In [100]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 10

# 7.1 Handling Missing Data

In [101]:
# missing data occurs commonly in many data analysis applications
# one of the goals of pandas is to make working with missing data as painless as possible

# missing data is represented with a NaN (not a number) value. This is called a sentinel value

# create data
string_data = pd.Series(['addvark', 'artichoke', np.nan, 'avocado'])
string_data

0      addvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [102]:
# detect NaN values using isnull() function
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [103]:
# in pandas, missing data is referred as NA (not available)

# Python's None value is also treated as NA in object arrays
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

## 7.1.1 Filtering Out Missing Data

In [104]:
# there are a few ways to filtering out missing data
# you can do it by hand using pd.isnull() and boolean indexing
# or you can use the dropna() method
from numpy import nan as NA

# on Series, it returns the Series with only the non-null data and index values
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna() # equivalent to: data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [105]:
# with DataFrame, you have to specify whether you want to drop rows or cols that are NA

data = pd.DataFrame([
    [1, 6.5, 3], 
    [1., NA, NA], 
    [NA, NA, NA], 
    [NA, 6.5, 3,]
])

data.dropna() # default: drops any row containing a missing value

,0,1,2
0,1.0,6.5,3.0


In [106]:
# passing how='all' will only drop rows that are all NA
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [107]:
# add another column
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [108]:
# to drop columns, pass axis=1 (or axis='columns')
data.dropna(axis='columns', how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [109]:
# a related way to filter out DataFrame rows tend to concern time series data (more on that later)

# create data
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA

df

,0,1,2
0,1.007189,NaN,NaN
1,0.228913,NaN,NaN
2,-2.001637,NaN,1.669025
3,-0.438570,NaN,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [110]:
df.dropna()

,0,1,2
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [111]:
# use the thresh=N argument to keep only rows containing at least N non NA values
df.dropna(thresh=2)

,0,1,2
2,-2.001637,NaN,1.669025
3,-0.438570,NaN,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


## 7.1.2 Filling In Missing Data

In [112]:
# rather than filtering out missing data, you may want to fill in the "holes" in any number of ways
# for most purposes, the fillna() function is the workhorse function to use

df.fillna(0) # fill all NA values with 0

,0,1,2
0,1.007189,0.000000,0.000000
1,0.228913,0.000000,0.000000
2,-2.001637,0.000000,1.669025
3,-0.438570,0.000000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [113]:
# calling fillna() with a dict will allows filling different values for each column

df.fillna({1 : 0.5, 2 : 0})

,0,1,2
0,1.007189,0.500000,0.000000
1,0.228913,0.500000,0.000000
2,-2.001637,0.500000,1.669025
3,-0.438570,0.500000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [114]:
# fillna() returns a new object, but you can modify the existing object in-place
_ = df.fillna(0, inplace=True)

df # df got changed

,0,1,2
0,1.007189,0.000000,0.000000
1,0.228913,0.000000,0.000000
2,-2.001637,0.000000,1.669025
3,-0.438570,0.000000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [115]:
# create data
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA

df

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,NaN,0.332883
3,-2.359419,NaN,-1.541996
4,-0.970736,NaN,NaN
5,0.377984,NaN,NaN


In [116]:
# the same interpolation methods available for reindexing can be used with fillna()

df.fillna(method='ffill') # fill the NA values with the 'nearest' non-NA value

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,0.560145,0.332883
3,-2.359419,0.560145,-1.541996
4,-0.970736,0.560145,-1.541996
5,0.377984,0.560145,-1.541996


In [117]:
# you can also add a 'limit' argument

df.fillna(method='ffill', limit=2) # only replace 2 NA values

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,0.560145,0.332883
3,-2.359419,0.560145,-1.541996
4,-0.970736,NaN,-1.541996
5,0.377984,NaN,-1.541996


In [118]:
# with fillna() you can do alot of things
# for example you might pass the mean or median value of a Series

data = pd.Series([1., NA, 3.5, NA, 7.])

data.fillna(data.mean()) # replace missing values with the mean of the dataset

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

# 7.2 Data Transformation

In [119]:
# we will learn how to process values, axes, and names

## 7.2.1 Removing Duplicates

In [120]:
# duplicate rows may be found in a DataFrame for any number of reasons. Example:

# create data
data = pd.DataFrame({
    'k1' : ['one', 'two'] * 3 + ['two'],
    'k2' : [1, 1, 2, 3, 3, 4, 4]
})

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [121]:
# DataFrame's method duplicated() returns a boolean Series indicating whether each row is a duplicate or not

data.duplicated() # the first duplicate value is still False (5 is false, but 6 is True)

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [122]:
# drop_duplicates() returns a DataFrame where the duplicated array is False

data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [123]:
# add another column

data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [124]:
# both of these methods by default consider all columns
# alterantively, you can specify any subset of them to detect duplicates

data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [125]:
# duplicated() and drop_duplicates() by default keep the first observed value combination
# passing keep='last' will return the last one

data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## 7.2.2 Transforming Data Using a Function or Mapping

In [126]:
# For many datasets, you may wish to perform some transformation 
# based on the values in the array, Series, or column in a DataFrame

# create data 
data = pd.DataFrame({
    'food' : ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'],
    'ounces' : [4, 3, 12, 6, 7.5, 8, 3, 5, 6]
})

data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [127]:
# suppose you wanted to add a column indicating the type of animal that each food came from
# let's write a mapping of each distinct meat type to the kind of animal

meat_to_animal = {
    'bacon' : 'pig',
    'pulled pork' : 'pig', 
    'pastrami' : 'cow',
    'corned beef' : 'cow',
    'honey ham' : 'pig',
    'nova lox' : 'salmon'
}

In [128]:
# here we have a problem: some of the meats are capitalized and others are not
# thus, we need to convert each value to lower case using the str.lower() Series method

lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [129]:
# the map() method on a Series accepts a function or dict-like object containing a mapping 

data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [130]:
# we could also have passed a function that does all the work

data['food2'] = data['food'].map(lambda x : meat_to_animal[x.lower()])
data # same result

,food,ounces,animal,food2
0,bacon,4.0,pig,pig
1,pulled pork,3.0,pig,pig
2,bacon,12.0,pig,pig
3,Pastrami,6.0,cow,cow
4,corned beef,7.5,cow,cow
5,Bacon,8.0,pig,pig
6,pastrami,3.0,cow,cow
7,honey ham,5.0,pig,pig
8,nova lox,6.0,salmon,salmon


## 7.2.3 Replacing Values

In [131]:
# Filling in missing data with the fillna() method is a special case of more general value replacement
# As you already seen, map() can be used to modify a subset of values in an object
# but replace() provides a simpler and more flexible way to do so

data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [132]:
# the -999. values might be sentinel values for missing data
# to replace these with NA values, we can use replace()
data.replace(-999, NA)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [133]:
# if you want to replace multiple values at once, you instead pass a list and then the substitue value

data.replace([-999, -1000], NA)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [134]:
# to use a different replacement for each value, pass a list of substitute
data.replace([-999, -1000], [NA, 0])

# the argument passed can also be a dict
# data.replace({-999:NA, -1000, 0}) # works the same

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## 7.2.4 Renaming Axis Indexes

In [135]:
# axis labels can be transformed by a function or mapping

data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index = ['Ohio', 'Colorado', 'New York'],
                    columns = ['one', 'two', 'three', 'four']
                   )

data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [136]:
# like Series, the axis indexes have a map() method

transform = lambda x : x[:4].upper().strip()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW'], dtype='object')

In [137]:
# you can assign to index to modify the DataFrame in-place

data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [138]:
# notably, rename() can be used in conjunction with a dict-like object providing new values for a subset of the axis labels

data.rename(index = {'OHIO' : 'INDIANA'},
            columns = {'three' : 'peekaboo', 'one' : 'bruh'})

,bruh,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## 7.2.5 Discretization and Binning

In [139]:
# Continuous data is often discretized or otherwise separated into “bins” for analysis

ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

# lets divide these into bin of [18, 25], [26, 35], [36, 60], and >= 61
# to do so, we use pd.cut()
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)

cats # the object returned is a special Categorical object
# you can treat Categories as an array of strings indicate the bin name

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [140]:
# inspect the 'codes' attribute
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [141]:
# inspect the 'categories' attribute

cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [142]:
# count values in each bin

pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

In [143]:
# By default, left is open (exclusive), while right is closed (inclusive)
# you can change which side is closed py passing a bool in the 'right' argument

pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [144]:
# you can also pass your own bin names by passing a list or array to the 'labels' argument

group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [145]:
# if you pass in an integer number of bins to cut() instead of explitcit bin edges
# it will compute equal-length bins based on the minimum and maximum values in the data

data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.79, 0.97], (0.6, 0.79], (0.79, 0.97], (0.41, 0.6], (0.22, 0.41], ..., (0.6, 0.79], (0.22, 0.41], (0.41, 0.6], (0.22, 0.41], (0.22, 0.41]]
Length: 20
Categories (4, interval[float64, right]): [(0.22, 0.41] < (0.41, 0.6] < (0.6, 0.79] < (0.79, 0.97]]

In [146]:
# qcut(), bins the data based on sample quantiles
# depending on the distribution of the data, using cut() will not usually result in each bin having the same number of data points
# but qcut() will help you obtain roughly equal-size bins

data = np.random.randn(1000) # normally distributed
cats = pd.qcut(data, 4) # cut into 4 quartiles

cats

[(0.62, 3.928], (-2.9499999999999997, -0.684], (0.62, 3.928], (0.62, 3.928], (0.62, 3.928], ..., (-2.9499999999999997, -0.684], (0.62, 3.928], (-0.0265, 0.62], (-0.0265, 0.62], (-2.9499999999999997, -0.684]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0265] < (-0.0265, 0.62] < (0.62, 3.928]]

In [147]:
pd.value_counts(cats)

(-2.9499999999999997, -0.684]    250
(-0.684, -0.0265]                250
(-0.0265, 0.62]                  250
(0.62, 3.928]                    250
dtype: int64

In [148]:
# similar to cut, you can pass in your own quantiles (numbers between 0 and 1, inclusive)
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1])

[(1.287, 3.928], (-2.9499999999999997, -1.187], (-0.0265, 1.287], (-0.0265, 1.287], (-0.0265, 1.287], ..., (-2.9499999999999997, -1.187], (-0.0265, 1.287], (-0.0265, 1.287], (-0.0265, 1.287], (-1.187, -0.0265]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -1.187] < (-1.187, -0.0265] < (-0.0265, 1.287] < (1.287, 3.928]]

## 7.2.6 Detecting and Filtering Outliers

In [149]:
# filtering or transforming outliers is largely a matter of applying array operations

data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.045383,0.024079,-0.004794,-0.050389
std,0.997796,1.008737,0.996599,0.996706
min,-3.645860,-3.184377,-3.745356,-3.428254
25%,-0.600254,-0.612162,-0.697084,-0.747478
50%,0.040150,-0.013609,-0.029924,-0.088274
75%,0.746527,0.690847,0.699046,0.623331
max,2.653656,3.525865,2.735527,3.366626


In [150]:
# suppose you want to find values in one of the columns exceeding 3 in absolute value

col = data[2]
col[np.abs(col) > 3]

38    -3.399312
133   -3.745356
Name: 2, dtype: float64

In [151]:
# to select all rows having a value exceeding 3 or -3, you can use the any() method on a boolean DataFrame

data[(np.abs(data) > 3).any(1)] 

# any(0) returns a Series with the index = the df columns
# any(1) returns a Sêries ưith the index = the df index 

,0,1,2,3
38,0.457246,-0.025907,-3.399312,-0.974657
57,1.951312,3.260383,0.963301,1.201206
133,0.508391,-0.196713,-3.745356,-1.520113
232,-0.242459,-3.056990,1.918403,-0.578828
255,0.682841,0.326045,0.425384,-3.428254
319,1.179227,-3.184377,1.369891,-1.074833
541,-3.548824,1.553205,-2.186301,1.277104
632,-0.578093,0.193299,1.397822,3.366626
779,-0.207434,3.525865,0.283070,0.544635
800,-3.645860,0.255475,-0.549574,-1.907459


In [152]:
# to cap values outside the interval to [-3, 3]

data[np.abs(data) > 3] = np.sign(data) * 3
data.describe() # min = -3 and max = 3

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.046577,0.023534,-0.003650,-0.050328
std,0.993777,1.005496,0.992788,0.994152
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.600254,-0.612162,-0.697084,-0.747478
50%,0.040150,-0.013609,-0.029924,-0.088274
75%,0.746527,0.690847,0.699046,0.623331
max,2.653656,3.000000,2.735527,3.000000


In [153]:
# np.sign() produces -1 or 1 based on whether the values in data are positive or negative

np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,1.0,-1.0
1,-1.0,1.0,-1.0,-1.0
2,1.0,-1.0,1.0,-1.0
3,1.0,-1.0,1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0


## 7.2.7 Permutation and Random Sampling

In [154]:
# permuting a Series or the rows in a DataFrame is easy to do using the numpy.random.permutation() function
# calling permutation() with the length of the axis you want to permute produces an array of ints indicating the new ordering
# that array can then be used in iloc[] or the equivalent take() function

df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))

sampler = np.random.permutation(5)
sampler

array([2, 4, 1, 0, 3])

In [155]:
# display data
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [156]:
df.take(sampler)
# df.iloc[sampler] # works the same

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15


In [157]:
# to select a random subset without replacement, you can use the sample method on Series and DataFrame

df.sample(n=3)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
1,4,5,6,7


In [158]:
# to generate a sample WITH replacement (to allow repeat choices), pass replace=True to sample()

choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True) # if replace=False, then n <= len(choices) or Error
draws

4    4
1    7
2   -1
3    6
4    4
1    7
4    4
0    5
3    6
0    5
dtype: int64

## 7.2.8 Computing Indicator/Dummy Variables

In [159]:
# another type of transformation for statistical modeling or machine learning application
# is converting a categorical variable into a "dummy" or "indicator" matrix

# pandas has get_dummies() function for doing this

df = pd.DataFrame({
                    'key' : ['b', 'b', 'a', 'c', 'a' , 'b'], 
                    'data1' : range(6)
                 })
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [160]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [161]:
# in some cases, you may want to add a prefix to the columns in the indicator DataFrame
# which can then be merged with other data
# get_dummies() has a 'prefix' argument just for that

dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummies = df[['data1']].join(dummies)
df_with_dummies

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [162]:
# if a row in a DataFrame belongs to multiple categories, things are a bit more complicated

# create data
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::', header=None, names=mnames, engine='python')
movies.head(10)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [163]:
# adding indicator for each genres requires a little bit of wrangling

# first, we extract the list of unique genres in the dataset
all_genres = []
for x in movies['genres']:
    all_genres.extend(x.split('|'))

genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [164]:
# one way to construct the indicator DataFrame is to start with a DataFrame of all zeros
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres) # full zeros df

# now, iterate through each movie and set entries of dummies to 1
# to do this, we use the dummies.columns to compute the column indices for each genres
# then use .iloc[] to set values
for i, gen in enumerate(movies['genres']):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

# then, as before, you can combine this with the original df
movie_with_indicator = movies.join(dummies.add_prefix('Genre_'))
movie_with_indicator.head(5)

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [165]:
# a useful recipe for statistical application is to combine get_dummies() with a discretization function like cut()

# create data
np.random.seed(12345)
values = np.random.rand(10)

values = np.array(['%.4f' % x for x in values], dtype=np.float64)
values

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

In [166]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

# use cut() to create categorical data
# use get_dummies to add indicator to those data, and turn them into a DataFrame
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


# 7.3 String Manipulation

In [167]:
# Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing
# most text operations are made simple with the string object's built-in methods
# for more complex pattern matching and text manipulations, regular expression may be needed

# pandas adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data
# additionally handling the annoyance of missing data

## 7.3.1 String Object Methods

In [168]:
# in many application, built-in string methods are sufficient
# as an example, a comma-separated string can be broken into pieces with split()

val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

In [169]:
# split() is often combined with strip() to trim whitespace (including line breaks)

pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [170]:
# strings can be concatenated using the join() method

'::'.join(pieces)

'a::b::guido'

In [171]:
# using the 'in' keyword to detect a substring

'guido' in val

True

In [172]:
# using index() to locate a substring (raise error if can't find)

val.index(',')

1

In [173]:
# using find() to locate a substring (return -1 if can't find)

val.find(':')

-1

In [174]:
# count() returns the number of occurences of a particular substring

val.count(',')

2

In [175]:
# replace() will substitute occurences of one pattern for another
print('replace "," with "::" -> ', val.replace(',', '::'))

# it is also used to delete patterns, too (by passing an empty string)
print('delete all "," -> ', val.replace(',', ''))

replace "," with "::" ->  a::b:: guido
delete all "," ->  ab guido


## 7.3.2 Regular Expressions

In [176]:
# python's built-in 're' module is responsible for applying regular expressions to strings
# here we only present couple of examples
import re

# suppose we wanted to split a string with a variable of whitespace characters
# the regex describing one or more whitespace characters is '\s+'
text = 'foo    bar\t baz   \tqux'
re.split('\s+', text) # the regex is first compiled, then re.split() is applied

['foo', 'bar', 'baz', 'qux']

In [177]:
# you can compile the regex yourself with re.compile()

regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [178]:
# if you want to get a list of all patterns matching the regex, use re.findall()

regex.findall(text)

['    ', '\t ', '   \t']

In [179]:
# pre compile your regex is highly recommended

text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

# regex for detecting emails: [username]@[domain name].[domain suffix]

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [180]:
# re.findall() returns a list of all matches

regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [181]:
# re.search() returns a special 'Match object' for the first match in the text

m = regex.search(text)
print(m)

# the match object can only tell us the start and end position of the pattern in the string
text[m.start():m.end()]

<re.Match object; span=(5, 20), match='dave@google.com'>


'dave@google.com'

In [182]:
# regex.match() returns None, as it only match if the pattern occurs at the start of the string
# text starts with: "Dave " (unmatch)

print(regex.match(text))

None


In [183]:
# sub() returns a new string with occurences of the pattern replaced by a new string

print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [184]:
# if you want to find email addresses and simultaneously segment each address into its 3 components,
# put parentheses around the parts of the pattern to segment

pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [185]:
# a 'Match object' produces by this modified regex returns a tuple of pattern components with its groups() method

m = regex.match('wesm@bright.net') # returns a Match object
m.groups() # retrieve the tuple of component

('wesm', 'bright', 'net')

## 7.3.3 Vectorized String Functions in pandas

In [186]:
# create data
data = {
    'Dave' : 'dave@google.com',
    'Steve' : 'steve@gmail.com',
    'Rob' : 'rob@gmail.com',
    'Wes' : np.nan
       }

data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [187]:
# data usually contains missing values (which built-in functions and regex will fail to apply)

data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [188]:
# Series has array-oriented methods for string operations that skip NA values
# these are accessed through the 'str' attribute

data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [189]:
# regex can be used too, along with any re options like IGNORECASE

pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
data.str.findall(pattern, flags = re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [190]:
# to do vectorized element retrieval, use str.get() or index into the str attribute

matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [191]:
# using str.get()

data.str.get(0) # get all the first characters

Dave       d
Steve      s
Rob        r
Wes      NaN
dtype: object

In [192]:
# using indexing/slicing

data.str[:5] # get all the first 5 characters

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object